# Complete reconception of the dataset loading

## Priorities
- Reduce the number of chunks. Combining members in intake-esm makes everything a lot worse.


Should I completely ditch intake-esm? At this point I barely see an advantage.

The methodology for now looks like this:
1. Read all single datasets into a larger dict with intake-esm, but do not aggregate. `dataset` level preprocessing is applied here.
2. Detrend on the `dataset` level
3. Move datasets to a nested dictionary
4. Combine grid_labels by using xesmf
5. Parse metrics to get a full `member` dataset.

## Things still to fix.

- [x] How is the control run making it through the detrending? Might work if it has the same time convention. But I prob have to code an exception in there...
- [x] ProgressBar for the trend loading
- [x] ProgressBar for the dim split saving
- [x] Compare results for CanESM5 (I have some old files for that...
- [x] Show non unique entries (https://stackoverflow.com/questions/9835762/how-do-i-find-the-duplicates-in-a-list-and-create-another-list-with-them)
    - [x] UKESM has duplicate versions...whyyyyyy
- [x] **Why does the rechunking 'setup' take so long? Raise issue?**
- [x] Norwegian models dont all have lev_bounds. Investigate
    - [x] Regridding needs to ignore area and thckcello! (Norwegian models)
- [ ] Sort out the datasets with missing fields before
- [ ] missing values alignment should also include dz_t?
- [ ] All nan slope files
- [ ] MPI-HR has missing detrending fields
- [ ]  ddict_nested_metrics['CMIP.MOHC.UKESM1-0-LL.piControl.r1i1p1f2'] has a drift in the upper dzt cells..roi = {'x':180, 'y':130, 'lev':0}
- **GFDL weird naming issue need to be fixed upstream**
- [ ] reconstruct metrics if they are not there (or parse them from another run? E.g. piControl is missing the area...)
- [ ] The removal of files is incredibly slow. `shutil.rmtree()` takes hours e.g. for CM4

- [ ] Try to use nc files instead of zarr? Might take too long...

- ~~[ ] warnings.warn('Input array is not F_CONTIGUOUS. ' 'Will affect performance.') Can I fix this somehow? its xesmf stuff./~~ 

In [1]:
import dask
dask.__version__

'2020.12.0'

## Should always have this to be able to go back with the versions

In [2]:
!conda list

# packages in environment at /scratch/gpfs/jbusecke/conda_tigercpu/envs/aguadv_omz_busecke_2021:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       1_gnu    conda-forge
affine                    2.3.0                      py_0    conda-forge
aguadv-omz-busecke-2021   0.1.dev16+g64162be.d20210402           dev_0    <develop>
aiobotocore               1.2.0                    pypi_0    pypi
aiohttp                   3.7.3                    pypi_0    pypi
aioitertools              0.7.1                    pypi_0    pypi
appdirs                   1.4.4              pyh9f0ad1d_0    conda-forge
asciitree                 0.3.3                      py_2    conda-forge
astropy                   4.2              py38h497a2fe_1    conda-forge
async-timeout             3.0.1                    pypi_0    pypi
attrs                     20.3.0             pyh

In [8]:
%load_ext autoreload
%autoreload 2

import numpy as np
import dask
import time
import shutil
import warnings
import xarray as xr

import matplotlib.pyplot as plt
from fastjmd95 import rho
import fastjmd95

assert fastjmd95.__version__ >= "0.2.1"
# import fastjmd95 #quick fix? https://github.com/dask/distributed/issues/3450#issuecomment-584739501
# Nope
from multiprocessing.pool import ThreadPool
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt

from xgcm import Grid
from fastprogress.fastprogress import progress_bar

from xarrayutils.file_handling import (
    write,
    maybe_create_folder,
    file_exist_check,
    temp_write_split,
)
from xarrayutils.utils import remove_bottom_values


from aguadv_omz_busecke_2021.vertical_transformation import (
    xgcm_transform_wrapper as transform_wrapper,
)
from aguadv_omz_busecke_2021.datasets import woa13
from aguadv_omz_busecke_2021.utils import print_html
from aguadv_omz_busecke_2021.preprocessing import (
    mixed_layer_wrapper,
    save_and_reload_rechunker,
    preprocessing_wrapper,
    strip_encoding,
)
from aguadv_omz_busecke_2021.omz_tools import (
    volume_consistency_checks,
    plot_omz_results,
    omz_thickness,
)
from aguadv_omz_busecke_2021.cmip6_stash import (
    detrend,
    construct_cfdate,
    _get_calendar,
    cmip6_dataset_id,
    combined_preprocessing,
    load_single_datasets,
    nested_dataset_wrapper,
    match_and_detrend,
    combine_grid_labels,
    parse_metrics,
)

# !!! Replace with the new metric parsing...

# from euc_dynamics.file_handling import
# from euc_dynamics.cmip6_preprocessing_stash import cmip6_dataset_id

# from euc_dynamics.cmip6_preprocessing_stash import combined_preprocessing

# from euc_dynamics.o2_utils import omz_thickness

# from euc_dynamics.utils import (mask_pacific, member_treatment,load_time_averages,
#                                 force_datasets_on_same_grid, master_load, o2_models,
#                                 combine_grid_labels, pick_time)


# dask.config.set(pool=ThreadPool(4)) ### New
dask.config.set(pool=ThreadPool(32))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This should in some form move to xgcm

In [9]:
def transform_wrapper_again(
    ds_in,
    intensive_vars=[
        "thetao",
        "o2",
        "so",
        "agessc",
    ],
):
    # alternate name: transform mean wrapper
    # sigma_bins
    sigma_bins = np.array([0, 24.5, 26.5, 27.65, 100])

    # define variables to be averaged (intensive quantities)
    intensive_vars = [
        "thetao",
        "o2",
        "so",
        "agessc",
    ]  # add 'uo', 'agessc' etc?

    intensive_vars = [v for v in intensive_vars if v in ds_in.data_vars]

    for iv in intensive_vars:
        dz = (xr.ones_like(ds_in[iv]) * ds_in.dz_t).where(~np.isnan(ds_in[iv]))
        ds_in[iv] = ds_in[iv] * dz

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ds_out = transform_wrapper(
            ds_in,
            extensive_vars=["omz_thickness"] + intensive_vars,
            target=sigma_bins,
        )

    # reconvert the same variables
    dz = ds_out.dz_t
    for iv in intensive_vars:
        ds_out[iv] = ds_out[iv] / dz
    return ds_out

## Testing distributed processing. 
This doesnt work really well...At this point I have almost all models and its not worth it to pursue this further

In [10]:
# from dask_jobqueue import SLURMCluster
# from dask.distributed import Client
# cluster = SLURMCluster(
#     scheduler_options={"dashboard_address": 9999},
# )
# client = Client(cluster)
# client

Template to fire up dask workers:

```python
/scratch/gpfs/jbusecke/conda_tigercpu/envs/aguadv_omz_busecke_2021/bin/python -m distributed.cli.dask_worker tcp://10.36.18.123:45961 --nthreads 2 --nprocs 20 --memory-limit 9GB --name <name_xx> --nanny --death-timeout 60 --local-directory /tmp --interface op0
```

## Put out the observations in density data first

In [11]:
foldername = "density_remapping_obs"
ofolder = maybe_create_folder(f"../../data/processed/{foldername}")

/scratch/gpfs/jbusecke/conda_tigercpu/envs/aguadv_omz_busecke_2021/lib/python3.8/site-packages/xarrayutils/file_handling.py:118: UserWarning: Folder ../../data/processed/density_remapping_obs does already exist.
  warnings.warn(f"Folder {path} does already exist.", UserWarning)


In [7]:
woa = woa13().drop('time')
woa['sigma_0'] = (fastjmd95.rho(woa.so, woa.thetao, 0) - 1000)
woa = woa.rename({'thkcello':'dz_t'})
woa = woa.chunk({'lev':-1,'x':180})
woa['omz_thickness'] = omz_thickness(woa, o2_bins=o2_bins)

woa_sigma = transform_wrapper_again(woa, intensive_vars=[va for va in woa.data_vars if va not in ['omz_thickness']])

# confirm that the volume is conserved
with ProgressBar():
    vol_sigma = (woa_sigma.omz_thickness * woa_sigma.areacello).sum(['x','y','sigma_0']).load()
    vol_z = (woa.omz_thickness * woa.areacello).sum(['x', 'y', 'lev']).load()
    
assert (((vol_z - vol_sigma)/vol_z*100)<0.5).all()

fname = ofolder.joinpath('woa13.nc')
woa_sigma = write(woa_sigma, fname, overwrite=True)

/scratch/gpfs/jbusecke/conda_tigercpu/envs/aguadv_omz_busecke_2021/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/scratch/gpfs/jbusecke/conda_tigercpu/envs/aguadv_omz_busecke_2021/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunk

NameError: name 'warnings' is not defined

In [ ]:
woa_sigma.isel(sigma_0=2).o2.plot(robust=True)

## Now the models

In [12]:
foldername = "density_remapping_time_yearly_final"
ofolder = maybe_create_folder(f"../../data/processed/{foldername}")
tempfolder = maybe_create_folder(f"../../data/temp/scratch_temp/{foldername}")

# global parameters
o2_bins = np.array([10, 40, 60, 80, 100, 120])

/scratch/gpfs/jbusecke/conda_tigercpu/envs/aguadv_omz_busecke_2021/lib/python3.8/site-packages/xarrayutils/file_handling.py:118: UserWarning: Folder ../../data/processed/density_remapping_time_yearly_final does already exist.
  warnings.warn(f"Folder {path} does already exist.", UserWarning)
/scratch/gpfs/jbusecke/conda_tigercpu/envs/aguadv_omz_busecke_2021/lib/python3.8/site-packages/xarrayutils/file_handling.py:118: UserWarning: Folder ../../data/temp/scratch_temp/density_remapping_time_yearly_final does already exist.
  warnings.warn(f"Folder {path} does already exist.", UserWarning)


In [13]:
import intake
from aguadv_omz_busecke_2021.utils import cmip6_collection
col = intake.open_esm_datastore(cmip6_collection(zarr=True))

In [14]:
col.df['source_id'].unique()

array(['CNRM-ESM2-1', 'UKESM1-0-LL', 'GFDL-ESM4', 'GFDL-CM4', 'CanESM5',
       'CanESM5-CanOE', 'MPI-ESM1-2-HR', 'ACCESS-ESM1-5', 'MRI-ESM2-0',
       'MIROC-ES2L', 'IPSL-CM6A-LR', 'NorESM2-LM', 'NorESM2-MM',
       'MPI-ESM1-2-LR', 'MPI-ESM-1-2-HAM', 'NorESM1-F', 'NorCPM1'],
      dtype=object)

In [15]:
ddict = load_single_datasets(
    col,
    source_id=[
        "UKESM1-0-LL", # area not available for all members
        "MRI-ESM2-0", # 
        "NorESM2-LM",
        "NorESM2-MM",
        "GFDL-ESM4",
        "GFDL-CM4",
        "MIROC-ES2L", # done
        "CNRM-ESM2-1", # done bunch of members have missing thetao
        "CanESM5", # done
        "CanESM5-CanOE", # done
        "MPI-ESM1-2-HR", # done
        "ACCESS-ESM1-5", # done
        "IPSL-CM6A-LR", # done
        "NorESM1-F", #meh, dont need it
        "NorCPM1", #meh, dont need it
        "MPI-ESM-1-2-HAM", # done
        "MPI-ESM1-2-LR", # done
    ],
    grid_label=["gr", "gn"],
    experiment_id=["historical", "ssp585", "piControl"],
    table_id=["Omon", "Ofx"],
    variable_id=["thetao", "so", "o2", "mlotst", "agessc", "thkcello", "areacello"],
)

Dataframe size before picking latest version: 2086
Getting latest version...

Dataframe size after picking latest version: 2079

Done....


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.version.zstore'


/scratch/gpfs/jbusecke/conda_tigercpu/envs/aguadv_omz_busecke_2021/lib/python3.8/site-packages/cmip6_preprocessing/preprocessing.py:211: UserWarning: MIROC-ES2L: No units found for lev
  warnings.warn(f'{ds.attrs["source_id"]}: No units found for {co}')
/scratch/gpfs/jbusecke/conda_tigercpu/envs/aguadv_omz_busecke_2021/lib/python3.8/site-packages/cmip6_preprocessing/preprocessing.py:211: UserWarning: MIROC-ES2L: No units found for lev
  warnings.warn(f'{ds.attrs["source_id"]}: No units found for {co}')
/scratch/gpfs/jbusecke/conda_tigercpu/envs/aguadv_omz_busecke_2021/lib/python3.8/site-packages/cmip6_preprocessing/preprocessing.py:211: UserWarning: MIROC-ES2L: No units found for lev
  warnings.warn(f'{ds.attrs["source_id"]}: No units found for {co}')
/scratch/gpfs/jbusecke/conda_tigercpu/envs/aguadv_omz_busecke_2021/lib/python3.8/site-packages/cmip6_preprocessing/preprocessing.py:211: UserWarning: MIROC-ES2L: No units found for lev
  warnings.warn(f'{ds.attrs["source_id"]}: No units f

In [16]:
# Load all trends into dict

import pathlib
trendfolder = pathlib.Path('../../data/processed/linear_regression_time_zarr_multimember/')
trend_models = np.unique([ds.attrs['source_id'] for ds in ddict.values()])
flist = []
for tm in trend_models:
    flist = flist + list(trendfolder.glob(f'*{tm}*_trend.nc'))

total = len(flist)
progress = progress_bar(range(total))

trend_dict = {}
for i,path in enumerate(flist):
    key = path.stem
    ds = xr.open_mfdataset([path])
    # write the filename in the dataset
    ds.attrs.update({'filepath':str(path)})
    # exclude all nan slopes (why are these there in the first place?)
    if not np.isnan(ds.slope).all():
        trend_dict[key] = ds
    else:
        print(f"found all nan regression data for {path}")
    progress.update(i)
progress.update(total)

found all nan regression data for ../../data/processed/linear_regression_time_zarr_multimember/CMIP_CCCma_CanESM5_historical_Omon_gn_v20190429_r22i1p1f1_so_trend.nc
found all nan regression data for ../../data/processed/linear_regression_time_zarr_multimember/CMIP_CCCma_CanESM5_historical_Omon_gn_v20190429_r23i1p1f1_o2_trend.nc
found all nan regression data for ../../data/processed/linear_regression_time_zarr_multimember/CMIP_CCCma_CanESM5_historical_Omon_gn_v20190429_r25i1p1f1_so_trend.nc
found all nan regression data for ../../data/processed/linear_regression_time_zarr_multimember/CMIP_CCCma_CanESM5_historical_Omon_gn_v20190429_r21i1p1f1_o2_trend.nc
found all nan regression data for ../../data/processed/linear_regression_time_zarr_multimember/CMIP_CCCma_CanESM5_historical_Omon_gn_v20190429_r24i1p1f1_so_trend.nc
found all nan regression data for ../../data/processed/linear_regression_time_zarr_multimember/CMIP_CCCma_CanESM5_historical_Omon_gn_v20190429_r22i1p1f1_o2_trend.nc
found all 

In [17]:
ddict_detrended = match_and_detrend(
    ddict,
    trend_dict,
    verbose=True,
    pass_variables=["areacello", "thkcello", "mlotst"],
)

Manually Ingnored Variable: Passing CMIP.CCCma.CanESM5.historical.r16i1p2f1.Ofx.areacello.gn unaltered
Manually Ingnored Variable: Passing ScenarioMIP.CCCma.CanESM5-CanOE.ssp585.r3i1p2f1.Ofx.thkcello.gn unaltered
Manually Ingnored Variable: Passing CMIP.CCCma.CanESM5.historical.r36i1p2f1.Ofx.thkcello.gn unaltered
Manually Ingnored Variable: Passing CMIP.CCCma.CanESM5.historical.r5i1p2f1.Ofx.areacello.gn unaltered
Manually Ingnored Variable: Passing CMIP.CCCma.CanESM5-CanOE.historical.r1i1p2f1.Ofx.thkcello.gn unaltered
Manually Ingnored Variable: Passing CMIP.IPSL.IPSL-CM6A-LR.historical.r31i1p1f1.Ofx.areacello.gn unaltered
Manually Ingnored Variable: Passing ScenarioMIP.CCCma.CanESM5.ssp585.r12i1p2f1.Ofx.thkcello.gn unaltered
Manually Ingnored Variable: Passing CMIP.NCC.NorESM2-LM.piControl.r1i1p1f1.Ofx.thkcello.gr unaltered
Manually Ingnored Variable: Passing CMIP.CCCma.CanESM5.historical.r12i1p2f1.Ofx.thkcello.gn unaltered
Manually Ingnored Variable: Passing CMIP.MPI-M.MPI-ESM1-2-LR.

/projects/GEOCLIM/LRGROUP/jbusecke/projects/aguadv_omz_busecke_2021/aguadv_omz_busecke_2021/cmip6_stash.py:262: UserWarning: No match found for ['NorCPM1', 'Omon', 'gn', 'r19i1p1f1', 'so'].
  warnings.warn(f"No match found for {match_elements}.")


Manually Ingnored Variable: Passing ScenarioMIP.CNRM-CERFACS.CNRM-ESM2-1.ssp585.r1i1p1f2.Omon.mlotst.gn unaltered
Manually Ingnored Variable: Passing ScenarioMIP.MOHC.UKESM1-0-LL.ssp585.r3i1p1f2.Omon.mlotst.gn unaltered


/projects/GEOCLIM/LRGROUP/jbusecke/projects/aguadv_omz_busecke_2021/aguadv_omz_busecke_2021/cmip6_stash.py:262: UserWarning: No match found for ['NorESM2-MM', 'Omon', 'gn', 'r1i1p1f1', 'so'].
  warnings.warn(f"No match found for {match_elements}.")


Manually Ingnored Variable: Passing CMIP.NCC.NorCPM1.historical.r7i1p1f1.Ofx.areacello.gn unaltered
Manually Ingnored Variable: Passing ScenarioMIP.CCCma.CanESM5.ssp585.r23i1p2f1.Ofx.thkcello.gn unaltered
Manually Ingnored Variable: Passing ScenarioMIP.MPI-M.MPI-ESM1-2-LR.ssp585.r8i1p1f1.Omon.thkcello.gn unaltered
Manually Ingnored Variable: Passing ScenarioMIP.CCCma.CanESM5.ssp585.r3i1p2f1.Ofx.areacello.gn unaltered
Manually Ingnored Variable: Passing CMIP.MPI-M.MPI-ESM1-2-LR.historical.r3i1p1f1.Ofx.areacello.gn unaltered


...

Manually Ingnored Variable: Passing CMIP.MPI-M.MPI-ESM1-2-HR.historical.r1i1p1f1.Omon.mlotst.gn unaltered
Control run: Passing CMIP.MPI-M.MPI-ESM1-2-LR.piControl.r1i1p1f1.Omon.thetao.gn unaltered
Manually Ingnored Variable: Passing CMIP.CSIRO.ACCESS-ESM1-5.historical.r1i1p1f1.Omon.mlotst.gn unaltered
Manually Ingnored Variable: Passing CMIP.CCCma.CanESM5.historical.r7i1p2f1.Omon.mlotst.gn unaltered
Control run: Passing CMIP.MOHC.UKESM1-0-LL.piControl.r1i1p1f2.Omon.agessc.gn unaltered
Manually Ingnored Variable: Passing CMIP.CCCma.CanESM5.historical.r36i1p2f1.Omon.mlotst.gn unaltered
Manually Ingnored Variable: Passing ScenarioMIP.IPSL.IPSL-CM6A-LR.ssp585.r14i1p1f1.Ofx.areacello.gn unaltered
Manually Ingnored Variable: Passing CMIP.IPSL.IPSL-CM6A-LR.historical.r11i1p1f1.Omon.mlotst.gn unaltered
Manually Ingnored Variable: Passing ScenarioMIP.CCCma.CanESM5.ssp585.r17i1p2f1.Ofx.areacello.gn unaltered
Control run: Passing CMIP.MRI.MRI-ESM2-0.piControl.r1i1p1f1.Omon.thetao.gn unaltered
Manu

/projects/GEOCLIM/LRGROUP/jbusecke/projects/aguadv_omz_busecke_2021/aguadv_omz_busecke_2021/cmip6_stash.py:262: UserWarning: No match found for ['MPI-ESM-1-2-HAM', 'Omon', 'gn', 'r2i1p1f1', 'so'].
  warnings.warn(f"No match found for {match_elements}.")


Control run: Passing CMIP.IPSL.IPSL-CM6A-LR.piControl.r1i1p1f1.Omon.thetao.gn unaltered
Manually Ingnored Variable: Passing ScenarioMIP.MPI-M.MPI-ESM1-2-LR.ssp585.r2i1p1f1.Ofx.thkcello.gn unaltered
Control run: Passing CMIP.MPI-M.MPI-ESM1-2-LR.piControl.r1i1p1f1.Omon.agessc.gn unaltered
Manually Ingnored Variable: Passing CMIP.MPI-M.MPI-ESM1-2-LR.piControl.r2i1p1f1.Omon.thkcello.gn unaltered
Manually Ingnored Variable: Passing ScenarioMIP.CCCma.CanESM5.ssp585.r14i1p2f1.Omon.mlotst.gn unaltered
Manually Ingnored Variable: Passing CMIP.HAMMOZ-Consortium.MPI-ESM-1-2-HAM.historical.r1i1p1f1.Ofx.areacello.gn unaltered
Manually Ingnored Variable: Passing CMIP.MPI-M.MPI-ESM1-2-HR.historical.r7i1p1f1.Ofx.thkcello.gn unaltered
Manually Ingnored Variable: Passing ScenarioMIP.MPI-M.MPI-ESM1-2-LR.ssp585.r8i1p1f1.Omon.mlotst.gn unaltered
Control run: Passing CMIP.CSIRO.ACCESS-ESM1-5.piControl.r1i1p1f1.Omon.so.gn unaltered
Manually Ingnored Variable: Passing CMIP.CNRM-CERFACS.CNRM-ESM2-1.historical.

/projects/GEOCLIM/LRGROUP/jbusecke/projects/aguadv_omz_busecke_2021/aguadv_omz_busecke_2021/cmip6_stash.py:262: UserWarning: No match found for ['MPI-ESM1-2-HR', 'Omon', 'gn', 'r4i1p1f1', 'so'].
  warnings.warn(f"No match found for {match_elements}.")
/projects/GEOCLIM/LRGROUP/jbusecke/projects/aguadv_omz_busecke_2021/aguadv_omz_busecke_2021/cmip6_stash.py:262: UserWarning: No match found for ['NorCPM1', 'Omon', 'gr', 'r17i1p1f1', 'thetao'].
  warnings.warn(f"No match found for {match_elements}.")


Manually Ingnored Variable: Passing CMIP.IPSL.IPSL-CM6A-LR.piControl.r1i2p1f1.Omon.mlotst.gn unaltered
Manually Ingnored Variable: Passing CMIP.IPSL.IPSL-CM6A-LR.piControl.r1i2p1f1.Omon.thkcello.gn unaltered
Control run: Passing CMIP.NCC.NorESM2-MM.piControl.r1i1p1f1.Omon.o2.gr unaltered
Control run: Passing CMIP.IPSL.IPSL-CM6A-LR.piControl.r1i1p1f1.Omon.o2.gn unaltered
Manually Ingnored Variable: Passing CMIP.CCCma.CanESM5-CanOE.piControl.r1i1p2f1.Omon.mlotst.gn unaltered
Manually Ingnored Variable: Passing CMIP.MOHC.UKESM1-0-LL.piControl.r1i1p1f2.Omon.thkcello.gn unaltered
Manually Ingnored Variable: Passing CMIP.CSIRO.ACCESS-ESM1-5.piControl.r1i1p1f1.Omon.mlotst.gn unaltered
Control run: Passing CMIP.IPSL.IPSL-CM6A-LR.piControl.r1i1p1f1.Omon.so.gn unaltered


In [18]:
ddict_detrended_filtered = {k:v for k,v in ddict_detrended.items() if 'rho' not in v.dims}

In [19]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    ddict_nested = nested_dataset_wrapper(ddict_detrended_filtered)

Could not find reference dataset for variables ['areacello' 'thkcello'] and grid_label gn
No datasets to merge in `combine_grid_labels`.
Could not find reference dataset for variables ['areacello' 'thkcello'] and grid_label gn
No datasets to merge in `combine_grid_labels`.
Could not find reference dataset for variables ['areacello' 'thkcello'] and grid_label gn
No datasets to merge in `combine_grid_labels`.
Could not find reference dataset for variables ['areacello' 'thkcello'] and grid_label gn
No datasets to merge in `combine_grid_labels`.
Could not find reference dataset for variables ['areacello' 'thkcello'] and grid_label gn
No datasets to merge in `combine_grid_labels`.
Could not find reference dataset for variables ['areacello' 'thkcello'] and grid_label gn
No datasets to merge in `combine_grid_labels`.
Could not find reference dataset for variables ['areacello' 'thkcello'] and grid_label gn
No datasets to merge in `combine_grid_labels`.
Could not find reference dataset for vari

In [ ]:
ddict_detrended_filtered['CMIP.NOAA']

## List the model/variables that had to be regridded

In [20]:
for k, ds in ddict_nested.items():
    hits = []
    if ds is not None:
        for va in ds.data_vars:
            if 'horizontally_regridded_from_grid_label' in ds[va].attrs.keys():
                hits.append(va)
        if len(hits)>0:
            print(f"regridded data for {k}:{hits}")

regridded data for CMIP.NCC.NorCPM1.piControl.r1i1p1f1.Omon:['so', 'thetao']
regridded data for CMIP.NCC.NorCPM1.piControl.r2i1p1f1.Omon:['so', 'thetao']
regridded data for CMIP.NCC.NorCPM1.piControl.r3i1p1f1.Omon:['so', 'thetao']
regridded data for CMIP.NCC.NorESM2-LM.historical.r1i1p1f1.Omon:['agessc', 'o2', 'so', 'thetao']
regridded data for CMIP.NCC.NorESM2-LM.historical.r2i1p1f1.Omon:['agessc', 'o2', 'so', 'thetao']
regridded data for CMIP.NCC.NorESM2-LM.historical.r3i1p1f1.Omon:['agessc', 'o2', 'so', 'thetao']
regridded data for CMIP.NCC.NorESM2-LM.piControl.r1i1p1f1.Omon:['agessc', 'o2', 'so', 'thetao']
regridded data for CMIP.NCC.NorESM2-MM.historical.r1i1p1f1.Omon:['agessc', 'o2', 'so', 'thetao']
regridded data for CMIP.NCC.NorESM2-MM.piControl.r1i1p1f1.Omon:['agessc', 'o2', 'so', 'thetao']
regridded data for CMIP.NOAA-GFDL.GFDL-CM4.historical.r1i1p1f1.Omon:['o2']
regridded data for CMIP.NOAA-GFDL.GFDL-CM4.piControl.r1i1p1f1.Omon:['mlotst', 'o2']
regridded data for CMIP.NOAA-G

In [21]:
ddict_nested_metrics = parse_metrics(ddict_nested)

/scratch/gpfs/jbusecke/conda_tigercpu/envs/aguadv_omz_busecke_2021/lib/python3.8/site-packages/cmip6_preprocessing/grids.py:358: UserWarning: Could not find the source_id/grid_label (MPI-ESM-1-2-HAM/gn) combo in `grid_dict`, returning `None`. Please submit an issue to github: https://github.com/jbusecke/cmip6_preprocessing/issues
  warnings.warn(
/scratch/gpfs/jbusecke/conda_tigercpu/envs/aguadv_omz_busecke_2021/lib/python3.8/site-packages/cmip6_preprocessing/grids.py:417: UserWarning: Staggered Grid creation failed. Returning `None`
  warnings.warn("Staggered Grid creation failed. Returning `None`")


fail for CMIP.HAMMOZ-Consortium.MPI-ESM-1-2-HAM.historical.r1i1p1f1
fail for CMIP.HAMMOZ-Consortium.MPI-ESM-1-2-HAM.historical.r2i1p1f1
fail for CMIP.HAMMOZ-Consortium.MPI-ESM-1-2-HAM.piControl.r1i1p1f1


In [22]:
list(ddict_nested_metrics.keys())

['CMIP.CCCma.CanESM5-CanOE.historical.r1i1p2f1',
 'CMIP.CCCma.CanESM5-CanOE.historical.r2i1p2f1',
 'CMIP.CCCma.CanESM5-CanOE.historical.r3i1p2f1',
 'CMIP.CCCma.CanESM5-CanOE.piControl.r1i1p2f1',
 'CMIP.CCCma.CanESM5.historical.r10i1p1f1',
 'CMIP.CCCma.CanESM5.historical.r10i1p2f1',
 'CMIP.CCCma.CanESM5.historical.r11i1p1f1',
 'CMIP.CCCma.CanESM5.historical.r11i1p2f1',
 'CMIP.CCCma.CanESM5.historical.r12i1p1f1',
 'CMIP.CCCma.CanESM5.historical.r12i1p2f1',
 'CMIP.CCCma.CanESM5.historical.r13i1p1f1',
 'CMIP.CCCma.CanESM5.historical.r13i1p2f1',
 'CMIP.CCCma.CanESM5.historical.r14i1p1f1',
 'CMIP.CCCma.CanESM5.historical.r14i1p2f1',
 'CMIP.CCCma.CanESM5.historical.r15i1p1f1',
 'CMIP.CCCma.CanESM5.historical.r15i1p2f1',
 'CMIP.CCCma.CanESM5.historical.r16i1p1f1',
 'CMIP.CCCma.CanESM5.historical.r16i1p2f1',
 'CMIP.CCCma.CanESM5.historical.r17i1p1f1',
 'CMIP.CCCma.CanESM5.historical.r17i1p2f1',
 'CMIP.CCCma.CanESM5.historical.r18i1p1f1',
 'CMIP.CCCma.CanESM5.historical.r18i1p2f1',
 'CMIP.CCCma.

## First test of the detrended data

In [ ]:
# # quick detrending QC example
# roi = {'x':180, 'y':130, 'lev':30}
# plot_vars = ['thetao', 'so', 'o2']
# test_plot_dict_raw = ddict
# test_plot_dict_detrended = ddict_detrended


# for k in [kk for kk in test_plot_dict_detrended.keys() if 'historical' in kk]:
#     print(k)
#     if not any([s in k for s in plot_vars]):
#         print('k')
#         pass
#     else:
#         hist_detrended = test_plot_dict_detrended[k]
#         hist = test_plot_dict_raw[k]
        
#         var = hist.attrs['variable_id']
#         with ProgressBar():
#             hist[var].isel(**roi).resample(time='5AS').mean().plot(color='C0')
#             hist_detrended[var].isel(**roi).resample(time='5AS').mean().plot(color='C1')
            
#         try:
#             k_cc = k.replace('CMIP', 'ScenarioMIP').replace('historical', 'ssp585')
#             cc_detrended = test_plot_dict_detrended[k_cc]
#             cc = test_plot_dict_raw[k_cc]
#             cc[var].isel(**roi).resample(time='5AS').mean().plot(color='C0')
#             cc_detrended[var].isel(**roi).resample(time='5AS').mean().plot(color='C1')
#         except:
#             print(f'cc failed for {k}')
            
            
#         plt.title(k)
#         plt.show()

## Test the nested (and regridded) output

In [ ]:
# # ds_plot = ddict_nested_metrics['CMIP.MPI-M.MPI-ESM1-2-HR.historical.r1i1p1f1']
# ds_plot = ddict_nested_metrics['CMIP.MIROC.MIROC-ES2L.historical.r3i1p1f2']
# ds_plot

In [ ]:
# # roi = {'x':180, 'y':130, 'lev':30}

# roi = {'x':180, 'y':130, 'lev':20} # for medium res...
# # roi = {'x':300, 'y':250, 'lev':20} # for higher res...
# with ProgressBar():
#     ds_plot.isel(**roi).thetao.plot()

In [ ]:
# with ProgressBar():
#     ds_plot.isel(**roi).agessc.plot()

In [ ]:
# roi = {'x':180, 'y':130, 'lev':0}
# with ProgressBar():
#     ds_plot.isel(**roi).dz_t.plot()

In [ ]:
# ds_plot.isel(**roi).dz_t

## Cheap copy paste of the transformation code on the quick

In [ ]:
def resample_yearly(ds_in, freq="1AS"):
    # this drops some coordinates, so i need to convert them to data_vars and then reconvert
    time_coords = [
        co
        for co in list(ds_in.coords)
        if "time" in ds_in[co].dims and co not in ["time", "time_bounds"]
    ]
    ds_out = ds_in.reset_coords(time_coords).resample(time=freq).mean()
    ds_out = ds_out.assign_coords({co: ds_out[co] for co in time_coords})
    ds_out.attrs.update({k: v for k, v in ds_in.attrs.items() if k not in ["table_id"]})
    return ds_out


def align_missing(ds_in):
    """Make sure that nans in all fields of a dataset are consistent.
    Requires"""
    # Due to the interpolation between `gr` and `gn`, we have to make sure that all data variables are masked in the same way!

    ds_mask = ds_in
    # for generalization np.logical_or.reduce((x, y, z))https://stackoverflow.com/questions/20528328/numpy-logical-or-for-more-than-two-arguments
    combo_nanmask = np.logical_or(
        np.isnan(ds_mask.o2).all("time").load(),
        np.isnan(ds_mask.thetao).all("time").load(),
    )
    try:
        plt.figure()
        combo_nanmask.isel(lev=5).plot()
        plt.show()
    except:
        pass

    ds_out = ds_in.where(~combo_nanmask)

    return ds_out

In [ ]:
list(ddict_nested_metrics.keys())

In [ ]:
for name, ds in ddict_nested_metrics.items():
    required_vars = ["thetao", "so", "o2", "dz_t"]#"areacello", 
    if not all(
        [va in ds.variables for va in required_vars]
    ):
        print(f"!!!\nMissing Variables {name}\n!!!")
        print([va for va in list(ds.variables) if va in required_vars])
    else:
        print(f"@@@@@@@@@@@@@@@@@\nProcessing {name}\n@@@@@@@@@@@@@@@@@")

        dataset_id = f"{cmip6_dataset_id(ds)}_{ds.attrs['variant_label']}"
        filename = ofolder.joinpath(f"{dataset_id}.zarr")

        t0 = time.time()
        if file_exist_check(filename) and not overwrite:
            print("Skipping. File exists already.")
            ds_sigma_reloaded = xr.open_zarr(
                filename, use_cftime=True, consolidated=True
            )
            try:
                plot_omz_results(ds_sigma_reloaded)
            except Exception as e:
                print(f"Plotting failed with: {e}")
        else:

            print(f"Writing to {filename}")

            tempfilelist = []

            plt.figure()
            ds.isel(time=200, lev=10).thetao.plot()
            plt.show()

            ## make sure the variables have been detrended (control run should not be)

            for va in [vv for vv in ds.data_vars if vv != "mlotst"]:
                print(va)
                if ds.attrs["experiment_id"] == "piControl":
                    assert "detrended_with_file" not in ds[va].attrs.keys()
                else:
                    assert "detrended_with_file" in ds[va].attrs.keys()

            # make sure the area has not been interpolated. I think the thickness is ok?
            for va in ["areacello"]:
                if va in ds.variables:
                    assert (
                        "horizontally_regridded_from_grid_label" not in ds[va].attrs.keys()
                    )

            if ds.attrs["experiment_id"] == "piControl":
                ds = ds.isel(time=slice(-300 * 12, None))

            ds = preprocessing_wrapper(ds)
            
            print('First temp save')
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore")

                ds, tempfilelist_var = temp_write_split(
                    ds,
                    tempfolder,
                    method='dimension',
                    verbose=True,
                    split_interval=12*10 if len(ds.x) < 700 else 12*1, # 2 years worked fine with ESM4 but for CM4 I just reduced it again..
                    file_name_pattern='raw_intermediate'
                )
                tempfilelist.extend(tempfilelist_var)

            print("Rechunking incoming dataset")
            with ProgressBar():
                temppath = tempfolder.joinpath("rechunked_intermediate.zarr")
                ds = save_and_reload_rechunker(ds.astype(np.float32), path=temppath)
                tempfilelist.append(temppath)

            # I need to align.mask the thickness aswell!
            ds = ds.reset_coords(["dz_t"])
            print("masking all fields consistently")
            with ProgressBar():
                ds = align_missing(ds)

            print(f"Remove bottom values")
            ds = remove_bottom_values(ds)

            print(f"Remove ML")
            ds = mixed_layer_wrapper(ds)

            ds = ds.set_coords("dz_t")

            ds["omz_thickness"] = omz_thickness(
                ds, o2_bins=o2_bins
            )

            ds["sigma_0"] = (fastjmd95.rho(ds.so, ds.thetao, 0) - 1000)
            
            ds_sigma_monthly = transform_wrapper_again(ds)

            perc_difference, omz_perc_difference = volume_consistency_checks(
                ds, ds_sigma_monthly
            )

            print(
                f"Relative difference ocean vol: {abs(perc_difference).data}% | OMZ vol {abs(omz_perc_difference).data}%"
            )

            if (abs(perc_difference) > 0.1).any() or (
                abs(omz_perc_difference) > 0.25# Had to increase for ESM4 before most would pass 0.01
            ).any():
                print("Volume differences exceed threshold. NOT SAVING.")
                print('\x1b[31m"red"\x1b[0m')
            else:
                ds_sigma_yearly = strip_encoding(resample_yearly(ds_sigma_monthly))
                with warnings.catch_warnings():
                    warnings.filterwarnings("ignore")

                    ds_sigma_yearly, tempfilelist_var = temp_write_split(
                        ds_sigma_yearly,
                        tempfolder,
                        verbose=True,
                        method='dimension',
                        split_interval=3,
                    )
                    tempfilelist.extend(tempfilelist_var)

                # rechunk the resulting file for convenience
                ds_sigma_yearly = strip_encoding(
                    ds_sigma_yearly.chunk({"x": -1, "y": -1, "time": 2})
                )

                #################### write out results ########################
                with ProgressBar():
                    ds_sigma_reloaded = write(
                        ds_sigma_yearly,
                        filename,
                        overwrite=overwrite,
                        force_load=False,
                        check_zarr_complete=True,
                    )
            ###### delete temps ######
            for tf in tempfilelist:
                if tf.exists():
                    shutil.rmtree(tf)
            ##################### Verification plotting ##########################
            try:
                plot_omz_results(ds_sigma_reloaded)
            except Exception as e:
                print(f"Plotting failed with: {e}")
        plt.show()
        t1 = time.time()
        print(f"Time passed: {(t1-t0)/60} minutes")

## More distributed

- Can I avoid the weird stealing crash (with old dask version) by just running two constant nodes, no singing on of off of workers?

## Experimenting with distributed to speed up CM4 (did not really succeed)

### Notes

- I cant get this to work with distributed. It did work, and was wonderfully fast. But it always got stuck with one stalled worker (which wouldnt even dissapear when I killed the process manually). 
- So I updated the dask versions and then the computation would not even start (I think it was still churning but took forever to even start the computation on the last step)
- FANTASTIC. NOW THAT HAS BUSTED THE OTHER WORKFLOW TOO....I HATE THIS...

- OK I can use the old env to run this with the threaded scheduler (slow) or use distributed (as long as I make sure that workers do not get disconnected, which leads to that stalled 'stealing' behvior)
- First let me check if the brand new env solves this at all...